# FRC with single images
**IIT 2018-01**

In this notebook I test the single image FRC with the same Nikon Confocal files that Taka gave me, with three different pinhole sizes.

In [1]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage.interpolation import zoom

import supertomo.analysis.resolution.fourier_ring_correlation as frc
import supertomo.ui.frc_options as options
import supertomo.ui.plots.image as showim
import supertomo.data.io.read as read
import supertomo.ui.plots.frc as frcplots
import supertomo.processing.image as imops
import supertomo.analysis.resolution.analysis as fsc_analysis
import supertomo.data.iterators.fourier_ring_iterators as iterators



from supertomo.data.containers.fourier_correlation_data import FourierCorrelationDataCollection

import warnings
warnings.filterwarnings("ignore", message="Polyfit may be poorly conditioned")

data_dir = "/Users/sami/Documents/Work/Data/2018_08_09_Split"
args_list = [data_dir, '--bin-delta=8', '--frc-curve-fit-degree=4', 
            '--resolution-threshold-criterion=snr', '--resolution-snr-value=0.25']

args = options.get_frc_script_options(args_list)

print args

Namespace(d_angle=20, d_bin=8, d_extract_angle=5.0, debug=False, directory='/Users/sami/Documents/Work/Data/2018_08_09_Split', frc_curve_fit_degree=4, hanning=False, hollow_iterator=False, min_filter=False, pathout=None, resol_square=False, resolution_point_sigma=0.01, resolution_snr_value=0.25, resolution_threshold_criterion='snr', resolution_threshold_curve_fit_degree=3, resolution_threshold_value=0.5, use_splines=False)


## Run FRC 

There are three image pairs in the dataset Taka gave me, each named with a pattern *filename_pinholesize.nd2* / *filename_pinholesize_.nd2*. Therefore it's rather easy to load them automatically for batch processing. Let's run the FRC for every image pair and save the results to FourierCorrelationDataCollection() which is the internal data format for storing FRC data.

In [2]:
from math import sqrt 

# Get sorted list of filenames
files = sorted([f for f in os.listdir(args.directory) if f.endswith(".tiff")])

data = FourierCorrelationDataCollection()
spacing = (0.04, 0.04)

# Calculate FRC for every image pair
for idx, image_name in enumerate(files):
    image_path = os.path.join(args.directory, image_name)
    image = read.get_image(image_path)
    image.spacing = spacing
    
    image1, image2 = imops.checkerboard_split(image)
    iterator = iterators.FourierRingIterator(image1.shape, args.d_bin)
    frc_task = frc.FRC(image1, image2, iterator)
    data[idx] = frc_task.execute()
                 

analyzer = fsc_analysis.FourierCorrelationAnalysis(data, spacing[0]*sqrt(2), args)
results = analyzer.execute()

TypeError: unsupported operand type(s) for *: 'int' and 'FourierRingIterator'

## Plot results

Use Fourier Data Plotter to plot the FRC curves.

In [ ]:
plotter = frcplots.FourierDataPlotter(results)
plotter.plot_all()

The angle values here translate to pinhole sizes as follows: 

- 0 -> 0.4AU
- 1 -> 1.2AU
- 2 -> 2.0AU

I use minimum filtering to minimize the effects of the saturation bumps at the tail end of the FRC curves. Also the curve fit is weighted with the amount of correlation.  There's still oscilallation, but at least the decay part follows the data quite nicely. 